In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os.path
from scipy import stats

from bikewheelcalc import BicycleWheel, Hub, Rim

%matplotlib inline

In [ ]:
# Every row is a wheel+width+tension
data = pd.DataFrame()

for fname in glob('../data/buckling_tension_K2/CR18-700_36sp_inbound/[!~]*_info.csv'):
    # Each file is a hub width

    d = pd.read_csv(fname, index_col=[0])
    
    f_noext = os.path.splitext(fname)[0]
    
    # Read load-displacement data
    Pd = pd.read_csv(f_noext[:-4]+'data.csv')
    
    for t in Pd['Tension [N]'].unique():
        Pd_t = Pd[Pd['Tension [N]'] == t]
        
        K2_lin = 16*np.polyfit(Pd_t['displacement [mm]'], Pd_t['delta Load [N]'], 1)[0]
        K2_quad = 16*np.polyfit(Pd_t['displacement [mm]'], Pd_t['delta Load [N]'], 2)[1]
        
        data = data.append(d['Value'].append(pd.Series({'Tension [N]': t,
                                                        'K2_lin': K2_lin,
                                                        'K2_quad': K2_quad})),
                           ignore_index=True)

data = data.apply(pd.to_numeric, errors='ignore')

data = data.drop(data[data['Flange spacing'] < 25].index, axis=0)
data.head()

In [ ]:
G = 26e9
E = 69e9

for i, r in data.iterrows():
    w = BicycleWheel()
    w.rim = Rim(radius=r['rim_radius'], area=r['rim_mass']/(2700.*2*np.pi*r['rim_radius']),
                I11=r['rim_GJ']/G, I22=r['rim_EI22']/E, I33=r['rim_EI11']/E, Iw=0.,
                young_mod=E, shear_mod=G)
    
    w.hub = Hub(width1=(r['Flange spacing']/2. - 1.5)/1000, diam1=0.058)
    
    w.lace_radial(n_spokes=36, diameter=np.sqrt(4*r['spk_EA']/(np.pi*210e9)), young_mod=210e9)
    
    data.at[i, 'Tbar'] = r['Num spokes']*r['Tension [N]']*w.spokes[0].n[1]/(2*np.pi*r['rim_radius'])
    data.at[i, 'kuu_0'] = r['Num spokes']*w.spokes[0].calc_k()[0, 0] / (2*np.pi*r['rim_radius'])

In [ ]:
d = pd.read_csv('../data/buckling_tension_K2/CR18_36s_80mm_T560_deflection.txt',
                delim_whitespace=True, header=None, names=['Spoke', 'Deflection'])

d['theta'] = np.arange(0., 2*np.pi, 2*np.pi/18)

th = np.linspace(0., 2*np.pi, 100)

with plt.style.context(('seaborn-paper')):
    fig, ax = plt.subplots(figsize=(3, 3))
    
    ax.plot(d['theta'], d['Deflection'], 'C0^', label='deflection')
    ax.plot(d['theta'], d['Deflection'] - d['Deflection'][0]/2 * np.cos(d['theta']), 'C3o', label='symmetric part')
    ax.plot(th, d['Deflection'][0]/4 * (np.cos(2*th) + 1), 'k--', label=r'$\cos{ (2 \theta) }$')
    
    plt.legend(loc='lower center')
    
    ax.set_xlim([0, 2*np.pi])
    ax.set_xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
    ax.set_xticklabels(['\$-\pi\$','','0', '', '\$\pi\$'])
    
    ax.set_ylim([-2, 0.4])
    
    ax.set_ylabel('Lateral deflection [mm]')
    ax.set_xlabel(r'\$\theta\$')
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_tension/_python_4pt_bend_def.pdf')

In [ ]:
def conf_int(x, y, xx):
    mean_x = np.mean(x)
    n = len(x)                       # number of samples in dataset
    t = stats.t.ppf(1-0.025, n - 2)  # t-score for 95% confidence interval, 2-sided test
    fit = np.polyfit(x, y, 1)
    
    y_p = fit[0]*x + fit[1]
    s_err = np.sum((y - y_p)**2)   # sum of the squares of the residuals

    confs = t*np.sqrt((s_err/(n-2))*(1./n + (np.power((xx-mean_x),2)/
                                         ((np.sum(np.power(x,2)))-n*(np.power(mean_x,2))))))
                   
    return confs

In [ ]:
n_widths = data[data['Flange spacing'] > 25]['Flange spacing'].nunique()
cp = sns.color_palette('Reds', n_widths)

K2_T_slope = []

with plt.style.context(('seaborn-paper')):

    fig, ax = plt.subplots(figsize=(3, 3))
    
    for i, w in enumerate(sorted(set(data[data['Flange spacing'] > 25]['Flange spacing'].tolist()))):
        T = data[data['Flange spacing'] == w]['Tbar']/1000
        K2 = data[data['Flange spacing'] == w]['K2_quad']

        fit = np.polyfit(T, K2, 1)
        
        TT = np.linspace(0., 80, 100)
        K2_p = fit[0]*TT + fit[1]
        
        K2_T_slope.append(fit[0])
        
        c_int = conf_int(T, K2, TT)
        
        ax.plot(T, K2, 'o', color=cp[i], label='{0:.1f} mm'.format(w))
        ax.plot([0., (-fit[1]/fit[0])], [fit[1], 0.], '-', color=cp[i])
        
        # Plot 95% confidence interval
        ax.fill_between(TT, K2_p + c_int, K2_p - c_int,
                        color=cp[i], alpha=0.3)
        
    ax.axis([0., 80., 0., 500.])
        
    ax.set_xlabel(r'\$\bar{T}\$ [N/mm]')
    ax.set_ylabel(r'\$n=2\$ Mode stiffness [N/mm]')

    ax.legend(loc='best')

    plt.tight_layout()
    plt.savefig('../figs/buckling_tension/_python_K2_vs_tension.pdf')
    
print np.mean(K2_T_slope), stats.sem(K2_T_slope)

# K2_0 vs. hub width

In [ ]:
data_widths = pd.DataFrame({'flange_spacing': data['Flange spacing']})

data['rim_radius']
data['d_hub'] = 0.058

data['eff_flange_width'] = data['Flange spacing'] - 3.0

K2_rim = (2*np.pi*82.0)**2 * data.iloc[0]['rim_mass'] / 2

print K2_rim / 1000

w_set = sorted(set(data[data['Flange spacing'] > 25]['Flange spacing'].tolist()))

x = np.zeros(len(w_set))
K2_0 = np.zeros(len(w_set))

for i, w in enumerate(w_set):
    data.at[data['Flange spacing'] == w, 'K2_0'] =\
        np.polyfit(data[data['Flange spacing'] == w]['Tension [N]'],
                   data[data['Flange spacing'] == w]['K2_quad'], 1)[1]

g = data.groupby(by='Flange spacing').mean()

x = g['kuu_0']*np.pi*g['rim_radius'] / 1000
y = g['K2_0']

with plt.style.context(('seaborn-paper')):

    fig, ax = plt.subplots(figsize=(3, 3))

    # Plot data points
    ax.plot(x, y, 'o')
    
    # Plot fit and confidence interval
    xx = np.linspace(0., 1000., 100)
    fit = np.polyfit(x, y, 1)
    K2_p = fit[0]*xx + fit[1]
    
    c_int = conf_int(x, y, xx)
    
    ax.plot([xx[0], xx[-1]], [K2_p[0], K2_p[-1]], 'C0-')
    
    ax.fill_between(xx, K2_p + c_int, K2_p - c_int,
                    color='C0', alpha=0.3)
    
    # Plot rim stiffness
    ax.plot([0., 1000.], [K2_rim/1000, K2_rim/1000], 'r--')
    
    # Plot theoretical
    ax.plot([0., 1000.], [K2_rim/1000, K2_rim/1000 + 1000], 'k--')

    ax.set_xlim([0, 750.])
    ax.set_xticks(range(0, 751, 250))
    ax.set_ylim([0., 500.])
    ax.set_xlabel(r'\$\pi R \kuu$ [N/mm]')
    ax.set_ylabel('Zero-tension stiffness [N/mm]')

    plt.tight_layout()
    plt.savefig('../figs/buckling_tension/_python_K2_0_spokes.pdf')

In [ ]:
import statsmodels.formula.api as smf

data['Tbar2'] = data['Tbar'] / 1000
data['K2_shift'] = data['K2_quad'] - data['K2_0']

lm = smf.ols(formula='K2_shift ~ Tbar2', data=data).fit()

lm.summary()